In [ ]:
# 라이브러리 임포트
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

print("="*80)
print("HTP 심리검사 해석 AI - 대화형 테스트")
print("="*80)

In [ ]:
# 설정
MODEL_PATH = "./qwen2.5-htp-layer-freeze-final"

print(f"모델 경로: {MODEL_PATH}")

## 모델 로드

In [ ]:
print("모델 로드 중...")

try:
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_PATH,
        device_map="auto",
        trust_remote_code=True,
        torch_dtype=torch.bfloat16,
    )
    
    tokenizer = AutoTokenizer.from_pretrained(
        MODEL_PATH,
        trust_remote_code=True,
        padding_side="right",
    )
    
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    
    print(f"✅ 모델 로드 완료!\n")
    print(f"Device: {model.device}")
    print(f"VRAM: {torch.cuda.memory_allocated() / 1e9:.2f} GB")
    
except Exception as e:
    print(f"❌ 모델 로드 실패: {e}")
    print(f"   경로 확인: {MODEL_PATH}")
    raise

## 추론 함수 정의

In [ ]:
def generate_interpretation(image_caption, max_tokens=256):
    """HTP 이미지 캡션을 받아 심리학적 해석 생성"""
    
    messages = [
        {
            "role": "system",
            "content": "You are an expert psychologist specialized in HTP (House-Tree-Person) test interpretation. "
                      "Provide detailed and insightful psychological analysis based on the given image caption."
        },
        {
            "role": "user",
            "content": f"Please provide a psychological interpretation of the following HTP test image caption\n\n"
                      f"Image caption: {image_caption}"
        }
    ]
    
    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_tokens,
            temperature=0.8,
            top_p=0.95,
            top_k=40,
            do_sample=True,
            repetition_penalty=1.2,
            no_repeat_ngram_size=3,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )
    
    input_length = inputs['input_ids'].shape[1]
    generated_tokens = outputs[0][input_length:]
    response = tokenizer.decode(generated_tokens, skip_special_tokens=True)
    
    return response.strip()

print("✅ 추론 함수 준비 완료")

## 입력 예시

In [ ]:
print("\n📝 입력 예시:")
print("  - The tree is very tall with dense branches")
print("  - A small house with no windows")
print("  - The person is drawn with very long arms")
print("  - The tree has broken branches and no leaves")
print("  - A large house with many windows and a wide door\n")

## 테스트 예제 실행

미리 준비된 예제로 테스트해보세요.

In [ ]:
# 예제 1
caption1 = "The tree is very tall with dense branches"

print(f"🖼️  입력: {caption1}")
print("\n🤖 AI 분석 중...\n")

result1 = generate_interpretation(caption1, max_tokens=200)

print("="*80)
print("📋 심리학적 해석:")
print("="*80)
print(result1)
print("="*80)

In [ ]:
# 예제 2
caption2 = "A small house with no windows"

print(f"🖼️  입력: {caption2}")
print("\n🤖 AI 분석 중...\n")

result2 = generate_interpretation(caption2, max_tokens=200)

print("="*80)
print("📋 심리학적 해석:")
print("="*80)
print(result2)
print("="*80)

In [ ]:
# 예제 3
caption3 = "The person is drawn with very long arms"

print(f"🖼️  입력: {caption3}")
print("\n🤖 AI 분석 중...\n")

result3 = generate_interpretation(caption3, max_tokens=200)

print("="*80)
print("📋 심리학적 해석:")
print("="*80)
print(result3)
print("="*80)

## 직접 입력해보기

아래 셀의 `your_caption` 변수에 원하는 이미지 캡션을 입력하고 실행하세요!

In [ ]:
# 여기에 원하는 캡션을 입력하세요
your_caption = "The tree has broken branches and no leaves"  # ← 이 부분을 수정하세요

print(f"🖼️  입력: {your_caption}")
print("\n🤖 AI 심리학자가 분석 중...\n")

try:
    interpretation = generate_interpretation(your_caption, max_tokens=200)
    
    print("="*80)
    print("📋 심리학적 해석:")
    print("="*80)
    print(interpretation)
    print("="*80)
    print("\n✅ 해석 완료!")
    
except Exception as e:
    print(f"❌ 오류 발생: {e}")
    print("   다시 시도해주세요.")

## 여러 개 테스트하기

리스트에 여러 캡션을 추가하여 한 번에 테스트할 수 있습니다.

In [ ]:
# 테스트할 캡션 리스트
test_captions = [
    "A large house with many windows and a wide door",
    "The person has very small hands",
    "The tree trunk is very thick and strong",
    # 여기에 더 추가할 수 있습니다
]

print(f"\n총 {len(test_captions)}개의 캡션을 테스트합니다.\n")

for i, caption in enumerate(test_captions, 1):
    print("\n" + "="*80)
    print(f"테스트 {i}/{len(test_captions)}")
    print("="*80)
    print(f"🖼️  입력: {caption}")
    print("\n🤖 AI 분석 중...\n")
    
    try:
        result = generate_interpretation(caption, max_tokens=200)
        print("📋 심리학적 해석:")
        print("-"*80)
        print(result)
        print("-"*80)
    except Exception as e:
        print(f"❌ 오류: {e}")

print("\n\n✅ 모든 테스트 완료!")